## Purpose

This notebook will run simlations and save the output into a file in ./dat/ 

In [17]:
# Set matplotlib for plotting in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

from fooof import FOOOF
from fooof import FOOOFGroup
from fooof.analysis import *
import numpy as np
from utils.ratios import *
from fooof.utils import trim_spectrum
# Import some utilities for synthesizing some test data
from fooof.synth import gen_group_power_spectra, param_sampler, gen_power_spectrum

The next cell define a single simulated spectrum

In [18]:
#Creats a single control simulated spectrum (Theta, peak alpha, beta)
def gen_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 12)) #slow alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

We will create a function that will take an input and generate ratios from PSD with slope 0 to the provided argument in integer increments. We will use numpy save and numpy load to save the results into a .nyp file.

In [19]:
#This function will gather ratio data from slope 0 to slope (argument)
# Offset is assumed 0
def sim_vary_slope(slope):
    
    res = []
    
    for i in range(slope):
        bg = [0,i]
        
        #100 trails for each treatment and control sim
        freq, power, _ = gen_group_power_spectra(100, [1,50], bg,gen_sample(), nlvs=np.random.uniform(.005,.02)) 

        fg = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)
        
        fg.fit(freq, power)
        
        res.append(get_group_ratios(fg,[4,8],[15,30]))
    np.save('./dat/slope_data',res)
    

In [20]:
sim_vary_slope(5)

We should now have the data from the simulation saved into ./dat directory. The next cell opens that newly created data.

In [21]:
np.load('./dat/slope_data.npy')

array([[[6.41363876, 5.61739525, 6.1280555 , ..., 6.86010962,
         6.34926415, 6.41545354],
        [1.45071479, 1.44470621, 1.37379815, ..., 1.46874173,
         1.54792766, 1.66656592],
        [1.86202416, 1.63085669, 1.77911289, ..., 1.99164473,
         1.84333475, 1.86255103]],

       [[0.49016446, 0.48729106, 0.49006249, ..., 0.48428605,
         0.49311383, 0.48790313],
        [0.77881937, 0.71183976, 0.73860357, ..., 0.80660225,
         0.75542873, 0.75884989],
        [0.14230581, 0.1414716 , 0.14227621, ..., 0.14059917,
         0.14316208, 0.1416493 ]],

       [[0.55256456, 0.5536012 , 0.55822303, ..., 0.55306076,
         0.55560886, 0.55012843],
        [0.56329695, 0.61227694, 0.54292493, ..., 0.5970206 ,
         0.56209285, 0.62426793],
        [0.16042197, 0.16072293, 0.16206475, ..., 0.16056603,
         0.1613058 , 0.15971471]],

       [[0.54628671, 0.54835523, 0.54795905, ..., 0.5450997 ,
         0.54802992, 0.54825532],
        [0.91301605, 0.88128494, 0

np.save() should have saved our 3d-array into './dat/slope_data.npy'

The outter most level will be the simulations we ran, 5 in this case. One level deeper is different ratio calculations - we have 3. Lastly, the lowest level in the 3D array are the individual trials which we set to 100.